<a href="https://colab.research.google.com/github/wayamamo/Somersaulting_paper/blob/main/DLC_Analysis_WY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using the derived x y coordinates from DeepLabCut (DLC), this code computes all the parameters including Time, Body length, Body length speed, Basal disc distance, Basal disc speed, and Foot angle.**

**This code also computes the number and time location of a somersalt**

---


Derived values will be later used to compute parameters within each step of a somersault.



Loading Data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from google.colab import files     #upload the file in the colab
uploaded = files.upload()

filename = next(iter(uploaded))               # add the name of file you want to analyze
dl = pd.read_csv(filename)

Renaming Headers in csv

In [ ]:
# The top three rows are renamed for the purpose of analysis


dl = dl.drop(dl.index[[0,1]])
dl.columns = ['#', 'mtx', 'mty', 'mtl', 'mbx', 'mby', 'mbl', 'fox', 'foy', 'fol', 'bdx', 'bdy', 'bdl']
dl = dl.reset_index()
dl = dl.drop(columns = ['index', 'mtl', 'mbl', 'fol', 'bdl'])
dl.head(5)

## Convert all the numbers to float
dl = dl.astype(float)
dl.dtypes

print(dl)

Computation of parameters

In [ ]:
#1 Basal disc speed
total_r = int(dl.iloc[-1,0])     #Total number of row
dl2 = dl
dl2['BD_speed'] = dl['mtx'] + dl['mty']
dl2['BD_speed'][0] = 0
#print(dl2)
for i in range(0, total_r):
  dl['BD_speed'][i+1] = np.sqrt((((dl['bdx'][i])-(dl['bdx'][i+1]))**2)+(((dl['bdy'][i])-(dl['bdy'][i+1]))**2))

print(dl2)


#2 Basal disc cumulative distance traveled (not as accurate)
total_r = int(dl.iloc[-1,0])     #Total number of row
dl2 = dl
dl2['BD_dist'] = dl['mtx'] + dl['mty']
dl2['BD_dist'][0] = 0
#print(dl2)
for i in range(0, total_r):
  dl['BD_dist'][i+1] = dl['BD_dist'][i] + np.sqrt((((dl['bdx'][i])-(dl['bdx'][i+1]))**2)+(((dl['bdy'][i])-(dl['bdy'][i+1]))**2))

print(dl2)



# Body length
total_r = int(dl.iloc[-1,0])     #Total number of row
dl2 = dl
dl2['Body_length'] = dl['mtx'] + dl['mty']
#print(dl2)
for i in range(0, total_r + 1):
  dl['Body_length'][i] = np.sqrt((((dl['mtx'][i])-(dl['mbx'][i]))**2)+(((dl['mty'][i])-(dl['mby'][i]))**2)) + \
                       np.sqrt((((dl['mbx'][i])-(dl['fox'][i]))**2)+(((dl['mby'][i])-(dl['foy'][i]))**2)) + \
                       np.sqrt((((dl['fox'][i])-(dl['bdx'][i]))**2)+(((dl['foy'][i])-(dl['bdy'][i]))**2))
                       
print(dl2)





# Foot angle
total_r = int(dl.iloc[-1,0])     #Total number of row
dl2 = dl

dl2['ax'] = dl['mbx'] - dl['fox']
dl2['ay'] = dl['mby'] - dl['foy']
dl2['bx'] = dl['bdx'] - dl['fox']
dl2['by'] = dl['bdy'] - dl['foy']

dl2['Foot_angle'] = dl['mtx'] + dl['mty']

for i in range(0, total_r + 1):
  value = ((dl2['ax'][i])*(dl2['bx'][i]) + (dl2['ay'][i])*(dl2['by'][i]))/ \
                        (np.sqrt((dl['ax'][i])**2 + (dl['ay'][i])**2)* \
                         np.sqrt((dl['bx'][i])**2 + (dl['by'][i])**2))
  dl['Foot_angle'][i] = np.arccos(value) * 180 / np.pi
                       
print(dl2['Foot_angle'])


In [ ]:
# Throw out noise for the Body_length    (high pass filter)   # parameter that can be changed



total_r = int(dl.iloc[-1,0])     #Total number of row    

dl2['Filtered_BL'] = dl['mtx'] + dl['mty']
#print(dl2)
for i in range(0, total_r + 1):
  if dl['Body_length'][i] > 136:                   #parameter
    dl['Filtered_BL'][i] = dl['Filtered_BL'][i-1]

  elif dl['Body_length'][i] <= 136:
    dl['Filtered_BL'][i] = dl['Body_length'][i]


                    
plt.plot(dl2['#'],dl2['Filtered_BL'],marker= '', color ='purple', linewidth = 3, label = 'Filtered_BL')

In [ ]:
plt.plot(dl2['#'],dl2['Body_length'],marker= '', color ='olive', linewidth = 3, label = 'Body_length')   

In [ ]:
#calculate SST distances based on frames selected that are part of the SST behaviour

distance = pd.DataFrame(dl2['BD_speed'])
print(distance)
totaldistance = distance.iloc[0:25].sum()
speed = pd.DataFrame(dl2['BD_dist'])
#totalspeed = speed[4]-speed[7]
averageSpeed = totaldistance/(79-9) #costumize frames to counti
print(totaldistance)
print(averageSpeed)

In [ ]:
plt.plot(dl2['#'],dl2['BD_speed'],marker= '', color ='olive', linewidth = 3, label = 'BD_speed')

In [ ]:
plt.figure(1)
plt.subplot(211)
plt.plot(dl2['#'],dl2['bdx'],marker= '', color ='olive', linewidth = 3, label = 'bdx')
plt.legend()

plt.subplot(212)
plt.plot(dl2['#'],dl2['bdy'],marker= '', color ='blue', linewidth = 3, label = 'bdy')

plt.legend()
plt.show()

In [ ]:
plt.figure(1)
plt.subplot(211)
plt.plot(dl2['#'],dl2['bdx'],marker= '', color ='olive', linewidth = 3, label = 'bdx')
plt.legend()

plt.subplot(212)
plt.plot(dl2['#'],dl2['bdy'],marker= '', color ='blue', linewidth = 3, label = 'bdy')

plt.legend()
plt.show()

In [ ]:
#smooth the bdx and bdy and calculate the sBD_speed

dl2['sbdx'] = (dl2['bdx'].rolling(5).sum())/5
dl2['sbdy'] = (dl2['bdy'].rolling(5).sum())/5





# calculate sBD_speed
total_r = int(dl.iloc[-1,0])     #Total number of row
dl2 = dl
dl2['sBD_speed'] = dl['mtx'] + dl['mty']
dl2['sBD_speed'][0] = 0
#print(dl2)
for i in range(0, total_r):
  dl['sBD_speed'][i+1] = np.sqrt((((dl['sbdx'][i])-(dl['sbdx'][i+1]))**2)+(((dl['sbdy'][i])-(dl['sbdy'][i+1]))**2))

plt.figure(1)
plt.subplot(311)
plt.plot(dl2['#'],dl2['sbdx'],marker= '', color ='olive', linewidth = 3, label = 'sbdx')
plt.legend()


plt.subplot(312)
plt.plot(dl2['#'],dl2['sbdy'],marker= '', color ='blue', linewidth = 3, label = 'sbdy')
plt.legend()

plt.subplot(313)
plt.plot(dl2['#'],dl2['sBD_speed'],marker= '', color ='red', linewidth = 3, label = 'sBD_s')
plt.legend()


plt.show()

In [ ]:
# get location of sBD_speed

n = 0
dl2['sSST_peak'] = 0              #dl['mtx'][0] + dl['mty'][0]

for i in range(1, total_r+1):
  if 3 < dl2['sBD_speed'][i] < 30:             # 1st threshold, decide from the graph of the BD speed
    dl2['sSST_peak'][n] = i
    n = n+1

 
#print(dl2['SST_peak'])




# make lists for SST_peak 
# move out of data frame of pandas, and work on actual numbers


SST_1 = []

for i in range(0, total_r+1):
  if ( dl2['sSST_peak'][i] != 0) :
    SST_1.append(dl2['sSST_peak'][i])
  else:
    break


print(SST_1)





# Detecting the onset time for each event 


SST_new = []    # after extracting first number of the peak
time_len = 0
SST_timeL = []
for i in range(0, len(SST_1)):             #  this means start locatin 0 and stop location at 35  len() function include count 0
  if i == 0:
    SST_new.append(SST_1[0])

  elif i == (len(SST_1)-1):
    time_len = time_len + (SST_1[i] - SST_1[i-1])
    SST_timeL.append(time_len)

  elif  (SST_1[i] - SST_1[i-1]) < 20:
    time_len = time_len + (SST_1[i] - SST_1[i-1])

  elif  (SST_1[i] - SST_1[i-1]) >= 20  :
    SST_new.append(SST_1[i])
    SST_timeL.append(time_len)
    time_len = 0


print(SST_new)
print(SST_timeL)
print(len(SST_new))



Saving Files



In [ ]:
dl2.to_csv(filename +'_ANALYZED.csv', index=False)
files.download(filename +'_ANALYZED.csv')

Detect the number of Somersaulting and get **locations**

In [ ]:
# get location of BD_speed

n = 0
dl2['SST_peak'] = 0              #dl['mtx'][0] + dl['mty'][0]

for i in range(1, total_r+1):
  if  7 < dl2['BD_speed'][i] < 23:             # 1st threshold, decide from the graph of the BD speed
    dl2['SST_peak'][n] = i
    n = n+1

 
print(dl2['SST_peak'])




# make lists for SST_peak 
# move out of data frame of pandas, and work on actual numbers


SST_1 = []

for i in range(0, total_r+1):
  if ( dl2['SST_peak'][i] != 0) :
    SST_1.append(dl2['SST_peak'][i])
  else:
    break


print(SST_1)





# Detecting the onset time for each event 


SST_new = []    # after extracting first number of the peak
time_len = 0
SST_timeL = []
for i in range(0, len(SST_1)):             #  this means start locatin 0 and stop location at 35  len() function include count 0
  if i == 0:
    SST_new.append(SST_1[0])

  elif i == (len(SST_1)-1):
    time_len = time_len + (SST_1[i] - SST_1[i-1])
    SST_timeL.append(time_len)

  elif  (SST_1[i] - SST_1[i-1]) < 20:
    time_len = time_len + (SST_1[i] - SST_1[i-1])

  elif  (SST_1[i] - SST_1[i-1]) >= 20  :
    SST_new.append(SST_1[i])
    SST_timeL.append(time_len)
    time_len = 0


print(SST_new)
print(SST_timeL)




# Final filtering: if the time is too long (>10s) it is false thus thrown out
# 10 can be played around

SSTloc = []
SSTloc_time = []
SSTnumber = 0

for i in range(0, len(SST_new)):
   if SST_timeL[i] < 10:                   
    SSTloc.append(SST_new[i])
    SSTloc_time.append(SST_timeL[i])
  
 # elif SST_timeL[i] >= 10:
 #    SSTloc.append(SST_new[i])
 #    SSTloc_time.append(SST_timeL[i])   


# make sure that elongation event was not included

print(SSTloc)
print(SSTloc_time)
print(len(SSTloc))

for i in range(0, len(SSTloc)-1):

  print(SSTloc[i])
  if  ( dl2['Body_length'][SSTloc[i]] - dl2['Body_length'][(SSTloc[i])+ 1]) < 2:            # in not really contracting the foot thrown out       
    SSTloc.pop(i)
    SSTloc_time.pop(i)

SSTnumber = len(SSTloc)

print(SSTloc)
print(SSTloc_time)
print(SSTnumber)